In [82]:
import pandas as pd
import random
import numpy as np
import csv


# Datos de interacciones de usuarios con películas, 
# la best_movie_id es el id de la película vista en el mismo día con mejor calificación dada por el usuario
data = pd.read_parquet(r"Netflix_Prize_data\netflix_data_sample.parquet")

# Datos de películas, contiene el id de la película, el año y el título
def read_movie_titles(csv_file_path):

    list_of_movies = []


    # Open the CSV file for reading
    with open(csv_file_path, mode='r', newline='') as file:
        # Read the file line by line
        for line in file:
            # Strip any leading/trailing whitespace including newlines
            line = line.strip()
            
            # Split the line into exactly 3 parts: first three fields and the rest
            parts = line.split(',', 2)
            
            # Ensure that we have exactly 4 parts (the last part may contain commas)
            if len(parts) == 3:
                id_field = parts[0]
                name_field = parts[1]
                description_field = parts[2]
                
                list_of_movies.append((id_field, name_field, description_field))

            else:
                # Handle cases where the line does not contain enough commas
                print(f'Unexpected line format: {line}')

    movie_titles = pd.DataFrame(list_of_movies, columns=["movie_id", "year", "title"])

    return movie_titles

movie_titles = read_movie_titles(r"Netflix_Prize_data\movie_titles.csv")




class Environment:
    def __init__(self, data, movie_titles):
        # el ambiente tiene en memoria el dataframe con los datos de las interacciones de usuarios
        self.data = data

        # se escoge como punto de partida cualquier contenido movie_id
        self.initial_state = data.sample(1)["movie_id"].iloc[0]

        # se inicializa el estado actual con el estado inicial
        self.state = self.initial_state

        # strikes controla la terminación del episodio, 1 strikes y el episodio termina
        self.strikes = 0
        # recompensa por defecto si se llega a 1 strike, está fuera (out)
        self.reward_out = -10



    def reset_strikes(self):
        self.strikes = 0
       
    def is_terminal(self):
        return self.strikes == 1

    def get_current_state(self):
        """
        Retorna el estado actual (el movie_id actual)

        Returns
        -------
        int
            Identificador de la película actual (movie_id)
        """
        return self.state

    def get_movie_name(self, movie_id):
        """
        Retorna el nombre de la película dado el movie_id

        Parameters
        ----------
        movie_id : int
            Identificador de la película

        Returns
        -------
        str
            Nombre de la película
        """
        return movie_titles[movie_titles.movie_id == movie_id].title.iloc[0]


    def get_possible_actions(self, state):
        """
        Retorna las acciones posibles dado un estado (movie_id), 
        las acciones serán cambiar a otro contenido dentro de los candidatos, los cuales son el grupo de películas con mejor calificación
        media del usuario que ha calificado la película actual
        """
        # obtener las películas candidatas, puntuación media dadas por los usuarios que vieron el contenido actual
        candidates = data[data.movie_id == state].groupby("best_movie_id")["best_rating"].mean().sort_values(ascending=False)
        # filtro para que solo se escojan las películas con mayor rating dentro del grupo de candidatos
        candidates = candidates[(candidates==candidates.max())].index

        return candidates
    
    def do_action(self, action):
        
        reward = 0
        done = False

        if self.is_terminal():
            reward = self.reward_out
            done = True
        else:
            # se obtiene la calificación media de los usuarios que vieron el contenido a recomendar
            rating = self.data[self.data.movie_id == action].rating.mean()
            # se escala la recompensa en función de la calificación media
            reward = self.reward_scalation(rating)

            # si el contenido recomendado tiene una calificación media menor a 3, se considera un strike para el agente,
            # de lo contrario se reinician los strikes
            if reward<0:
                self.strikes += 1
            else:
                self.reset_strikes()

        # recordar que la acción es el cambio de contenido, el nuevo estado es el nuevo contenido (id_movie a recomendar)
        self.state = action
        return reward, self.state, done


    def reward_scalation(self, rating):
        # la escala retorna 1 si el rating medio es 5, 0 si es 3 y -1 si es 1
        return (rating-3)/2


    def reset(self):
        self.state = self.initial_state



# agente

        
class Agent:
    def __init__(self, env, gamma=0.9, alpha=0.1, epsilon=0.9, episodes=1000):

        # el agente tiene en memoria el ambiente
        self.environment = env
        # gamma factor de descuento
        self.gamma = gamma
        # alpha tasa de aprendizaje
        self.alpha = alpha
        # epsilon factor de exploración
        self.epsilon = epsilon
        # decay_rate tasa de decaimiento de epsilon
        self.decay_rate = 0.9

        # número de episodios
        self.episodes = episodes
        # Q table inicializada como un diccionario vacío, los estados y acciones se irán añadiendo a medida que se vayan explorando (evita crear una tabla muy grande)
        self.qtable ={}


    def run(self):

        for episode in range(self.episodes):
            self.environment.reset()
            # estado actual (movie_id actual)
            state = self.environment.get_current_state()
            # done indica si el episodio ha terminado (si el agente ha llegado a 3 strikes)
            done = False

            while not done:
                # se escoge una acción (cambio de contenido) al azar o con base en la Q table determinado por epsilon
                action = self.random_action(state)

                reward, next_state, done = self.step(action)
                
                # si el estado no está en la Q table, se añade
                if state not in self.qtable:
                    self.qtable[state] = {action: 0}
                else:
                    # si la acción no está en la Q table, se añade
                    if action not in self.qtable[state]:
                        self.qtable[state][action] = 0

                # valor del estado actual en la Q table
                old_value = self.qtable[state][action]
                # valor del estado siguiente en la Q table
                next_max = max(self.qtable[next_state].values()) if next_state in self.qtable else 0
                # cálculo del nuevo valor del estado actual con base en la ecuación de Bellman
                new_value = old_value + self.alpha * (reward + self.gamma * next_max - old_value)


                if action is not None:
                    self.qtable[state][action] = new_value
                else:
                    self.qtable[state][action] = reward
                    
                state = next_state
            
            self.epsilon = max(self.epsilon * self.decay_rate,0.01)


    def random_action(self, current_state):

        possible_actions = self.environment.get_possible_actions(current_state)

        # fase de exploración (adquirir conocimiento)
        if random.uniform(0, 1) < self.epsilon:
            return random.choice(possible_actions)
        
        # fase de explotación del conocimiento
        else:
            # si aún no tiene conocimiento del estado actual, se escoge una al azar    
            if ~ (current_state in self.qtable.keys()):
                best_action = random.choice(possible_actions)

            # si ya tiene conocimiento, se escoge la acción con mayor valor
            else:
                max_value = max(self.qtable[current_state].values())
                max_keys = [key for key, value in self.qtable[current_state].items() if value == max_value]
                best_action = random.choice(max_keys)

            return best_action



    
    def step(self, action):
       
        return self.environment.do_action(action)
    
    
    def actions_values(self):

        actions = {}
        values = {}
        recommended = {}
        for state, action_values in self.qtable.items():
            
            # id de la película recomendada con mayor valor en la Q table por cada estado
            max_action = max(action_values, key=action_values.get)
            actions[state] = max_action
            
            values[state] = max(action_values.values())
            
            # nombre de película actual
            actual_movie = self.environment.get_movie_name(state)
            recommended[actual_movie] = self.environment.get_movie_name(max_action)
        
        return actions, values, recommended
        

        
    # def max_action(self, current_state):
    #     action_index = np.argmax(self.qtable[current_state]) 
    #     actions = self.environment.actions
    #     return actions[action_index]

    # def action_name(self, action_index):
    #     return self.environment.actions[action_index]
    
    # def action_index(self, action):
    #     actions = self.environment.actions
    #     for i in range(len(actions)):
    #         if actions[i] == action:
    #             return i
    #     return -1



env = Environment(data, movie_titles)

#creación del agente
agent = Agent(env, gamma=0.9, alpha=0.1, epsilon=0.9, episodes=10) #episodios reducidos para efectos de demostración
#ejecución del agente
agent.run()
actions, values, recommended = agent.actions_values()

In [83]:
agent.qtable

{'5294': {'4356': 0.02860183366077276,
  '3962': -1.0,
  '62': -1.9,
  '2195': -1.0,
  '1299': -1.0,
  '334': -1.9,
  '17405': -1.0,
  '1645': -1.0},
 '4356': {'406': 0.017293777134587553},
 '406': {'700': 0.019921875000000002},
 '700': {'330': 0.012903225806451625},
 '330': {'12494': 0.024461057023643942},
 '12494': {'6974': 0.06683501683501683},
 '6974': {'16147': 0.06978798586572439},
 '16147': {'789': 0.03074034334763949},
 '789': {'2813': 0.009728506787330327},
 '2813': {'357': 0.007231149567367124},
 '357': {'12672': 0.030307406643284508},
 '12672': {'992': 0.03220338983050848},
 '992': {'5582': 0.07257942511346448},
 '5582': {'1329': 0.011158798283261807},
 '1329': {'12870': 0.06966934487021015},
 '12870': {'8644': 0.03666435345824659},
 '8644': {'6552': 0.005783267827063444},
 '6552': {'1703': 0.03660772757039948},
 '1703': {'629': 0.018279569892473126},
 '629': {'1865': 0.018806354658651794},
 '1865': {'1707': 0.025694444444444443},
 '1707': {'7786': 0.041438356164383565},
 '7

In [84]:
actions

{'5294': '4356',
 '4356': '406',
 '406': '700',
 '700': '330',
 '330': '12494',
 '12494': '6974',
 '6974': '16147',
 '16147': '789',
 '789': '2813',
 '2813': '357',
 '357': '12672',
 '12672': '992',
 '992': '5582',
 '5582': '1329',
 '1329': '12870',
 '12870': '8644',
 '8644': '6552',
 '6552': '1703',
 '1703': '629',
 '629': '1865',
 '1865': '1707',
 '1707': '7786',
 '7786': '1642',
 '1642': '12600',
 '12600': '4881',
 '4881': '5762',
 '5762': '331',
 '331': '12881',
 '12881': '14737',
 '14737': '2000',
 '2000': '17381',
 '17381': '11773',
 '11773': '10699',
 '10699': '405',
 '405': '4378',
 '4378': '6664',
 '6664': '97',
 '97': '2452',
 '2452': '12299',
 '12299': '2920',
 '2920': '341',
 '341': '798'}

In [85]:
values

{'5294': 0.02860183366077276,
 '4356': 0.017293777134587553,
 '406': 0.019921875000000002,
 '700': 0.012903225806451625,
 '330': 0.024461057023643942,
 '12494': 0.06683501683501683,
 '6974': 0.06978798586572439,
 '16147': 0.03074034334763949,
 '789': 0.009728506787330327,
 '2813': 0.007231149567367124,
 '357': 0.030307406643284508,
 '12672': 0.03220338983050848,
 '992': 0.07257942511346448,
 '5582': 0.011158798283261807,
 '1329': 0.06966934487021015,
 '12870': 0.03666435345824659,
 '8644': 0.005783267827063444,
 '6552': 0.03660772757039948,
 '1703': 0.018279569892473126,
 '629': 0.018806354658651794,
 '1865': 0.025694444444444443,
 '1707': 0.041438356164383565,
 '7786': 0.03958115183246074,
 '1642': 0.033814303638644926,
 '12600': 0.03089080459770115,
 '4881': 0.037370867768595044,
 '5762': 0.020854826823876207,
 '331': 0.018044747081712064,
 '12881': 0.006776180698151957,
 '14737': 0.027764026402640266,
 '2000': 0.04338129496402879,
 '17381': 0.025,
 '11773': 0.03947368421052631,
 '10

In [86]:
recommended

{'Tom and Jerry: Whiskers Away!': 'Road to Perdition',
 'Road to Perdition': 'Hostage',
 'Hostage': "Todd McFarlane's Spawn",
 "Todd McFarlane's Spawn": 'Wild Things',
 'Wild Things': 'Behind Enemy Lines',
 'Behind Enemy Lines': 'The Usual Suspects',
 'The Usual Suspects': 'The Sopranos: Season 1',
 'The Sopranos: Season 1': 'Boyz N the Hood',
 'Boyz N the Hood': 'Bad Education',
 'Bad Education': 'House of Sand and Fog',
 'House of Sand and Fog': 'John Q',
 'John Q': 'The Cowboys',
 'The Cowboys': 'Star Wars: Episode V: The Empire Strikes Back',
 'Star Wars: Episode V: The Empire Strikes Back': 'He Got Game',
 'He Got Game': "Schindler's List",
 "Schindler's List": 'Catch Me If You Can',
 'Catch Me If You Can': "Charlie's Angels",
 "Charlie's Angels": 'Ever After: A Cinderella Story',
 'Ever After: A Cinderella Story': 'Firestarter',
 'Firestarter': 'Eternal Sunshine of the Spotless Mind',
 'Eternal Sunshine of the Spotless Mind': "Outfoxed: Rupert Murdoch's War on Journalism",
 "Outf